# SQL: HỆ THỐNG QUẢN LÝ DOANH SỐ

In [ ]:
import pymysql
import os
from dotenv import load_dotenv

# Load environment variables from .credential file
load_dotenv(".credential")

# Get credentials from environment variables
conn = pymysql.connect(
    host=os.getenv("DB_HOST"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD")
)
cursor = conn.cursor()


Current working directory: f:\AIO\warm-up-25\warm-up1\w7\exercises


In [80]:
cursor.execute("CREATE DATABASE IF NOT EXISTS SalesDB;")
cursor.execute("USE SalesDB;")


# 1. Tạo bảng Salespersons
cursor.execute("""
CREATE TABLE IF NOT EXISTS SALESPERSONS (
    SalespersonID INT PRIMARY KEY,
    FullName VARCHAR(100) NOT NULL,
    Email VARCHAR(100) UNIQUE NOT NULL,
    PhoneNumber VARCHAR(15) NOT NULL        
);
""")

# 2. Tạo bảng CUSTOMERS
cursor.execute("""
CREATE TABLE IF NOT EXISTS CUSTOMERS (
    CustomerID INT PRIMARY KEY,
    FullName VARCHAR(100) NOT NULL,
    Email VARCHAR(100) UNIQUE NOT NULL,
    PhoneNumber VARCHAR(15) NOT NULL        
);
""")

# 3. Tạo bảng ORDERS
cursor.execute("""
CREATE TABLE IF NOT EXISTS ORDERS (
    OrderID INT PRIMARY KEY,
    CustomerID INT NOT NULL,
    SalespersonID INT NOT NULL,
    OrderDate DATE NOT NULL,
    TotalAmount DECIMAL(12, 2) NOT NULL,
    FOREIGN KEY (CustomerID) REFERENCES CUSTOMERS(CustomerID) ON DELETE CASCADE,
    FOREIGN KEY (SalespersonID) REFERENCES SALESPERSONS(SalespersonID) ON DELETE CASCADE

);
""")

# 4. Chèn dữ liệu vào bảng SALESPERSONS
cursor.execute("""
INSERT INTO SALESPERSONS (SalespersonID, FullName, Email, PhoneNumber) VALUES
(1, 'John Doe', 'john.doe@example.com', '123-456-7890'),
(2, 'Jane Smith', 'jane.smith@example.com', '234-567-8901'),
(3, 'Michael Brown', 'michael.brown@example.com', '345-678-9012'),
(4, 'Emily Johnson', 'emily.johnson@example.com', '456-789-0123'),
(5, 'David Wilson', 'david.wilson@example.com', '567-890-1234'),
(6, 'Sophia Turner', 'sophia.turner@example.com', '678-123-4567'),
(7, 'Daniel Harris', 'daniel.harris@example.com', '789-234-5678')
ON DUPLICATE KEY UPDATE FullName=VALUES(FullName), Email=VALUES(Email), PhoneNumber=VALUES(PhoneNumber);
""")

# 5. Chèn dữ liệu vào bảng CUSTOMERS
cursor.execute("""
INSERT INTO CUSTOMERS (CustomerID, FullName, Email, PhoneNumber) VALUES
(101, 'Alice Green', 'alice.green@example.com', '678-901-2345'),
(102, 'Bob White', 'bob.white@example.com', '789-012-3456'),
(103, 'Charlie Black', 'charlie.black@example.com', '890-123-4567'),
(104, 'Diana Blue', 'diana.blue@example.com', '901-234-5678'),
(105, 'Edward Grey', 'edward.grey@example.com', '012-345-6789'),
(106, 'Frank Orange', 'frank.orange@example.com', '345-678-9012'),
(107, 'Grace Yellow', 'grace.yellow@example.com', '456-789-0123')
ON DUPLICATE KEY UPDATE FullName=VALUES(FullName), Email=VALUES(Email), PhoneNumber=VALUES(PhoneNumber);
""")

# 6. Chèn dữ liệu vào bảng ORDERS
cursor.execute("""
INSERT INTO ORDERS (OrderID, CustomerID, SalespersonID, OrderDate, TotalAmount) VALUES
(1001, 101, 1, '2024-03-01', 250.75),
(1002, 102, 2, '2024-03-02', 499.99),
(1003, 103, 3, '2024-03-03', 320.50),
(1004, 104, 4, '2024-03-04', 150.00),
(1005, 105, 5, '2024-03-05', 275.80),
(1006, 101, 1, '2024-03-06', 199.99),  
(1007, 101, 2, '2024-03-07', 349.50),  
(1008, 102, 3, '2024-03-08', 120.00),  
(1009, 103, 1, '2024-03-09', 410.20),  
(1010, 104, 2, '2024-03-10', 530.00),  
(1011, 106, 6, '2024-03-11', 300.00),  
(1012, 107, 7, '2024-03-12', 450.75)
ON DUPLICATE KEY UPDATE CustomerID=VALUES(CustomerID), SalespersonID=VALUES(SalespersonID), OrderDate=VALUES(OrderDate), TotalAmount=VALUES(TotalAmount);
""")

# Commit the changes
conn.commit()


In [81]:
# 7. Liệt kê các đơn hàng có SalespersonID = 1
cursor.execute("""
SELECT * FROM ORDERS WHERE SalespersonID = 1;
""")

data = cursor.fetchall()
print(data)

((1001, 101, 1, datetime.date(2024, 3, 1), Decimal('250.75')), (1006, 101, 1, datetime.date(2024, 3, 6), Decimal('199.99')), (1009, 103, 1, datetime.date(2024, 3, 9), Decimal('410.20')))


In [82]:
# 8. Tìm khách hàng có số lượng đơn hàng nhiều nhất.
cursor.execute("""
SELECT CustomerID, count(*) AS order_times from orders group by CustomerID order by order_times desc limit 1;
""")

data = cursor.fetchall()
print(data)

((101, 3),)


In [83]:
# 9. Tính tổng doanh số của từng nhân viên.
cursor.execute("""
select SalespersonID, sum(TotalAmount) AS sales_person_revenue from orders group by SalespersonID;
""")

data = cursor.fetchall()
print(data)

((1, Decimal('860.94')), (2, Decimal('1379.49')), (3, Decimal('440.50')), (4, Decimal('150.00')), (5, Decimal('275.80')), (6, Decimal('300.00')), (7, Decimal('450.75')))


In [84]:
# 10.  Tìm nhân viên có doanh số cao nhất.
cursor.execute("""
select SalespersonID, sum(TotalAmount) AS sales_person_revenue from orders group by SalespersonID order by sales_person_revenue desc limit 1;
""")

data = cursor.fetchall()
print(data)

((2, Decimal('1379.49')),)


In [85]:
# 11: Cập nhật tổng giá trị đơn hàng có OrderID = 2 thành 4.500.000
cursor.execute("""
update orders set TotalAmount = 450000 where OrderID = 1002;
""")

conn.commit()


In [86]:
#  12: Cập nhật số điện thoại của khách hàng có CustomerID = 3
cursor.execute("""
update customers set PhoneNumber = '333-333-0000' where CustomerID = 103;
""")

conn.commit()


In [87]:
#  13: Xóa tất cả các đơn hàng của khách hàng có CustomerID = 2
cursor.execute("""
delete from orders where CustomerID = 102;
""")

conn.commit()


In [90]:
# # Get All Databases and Their Sizes
# cursor.execute("""
#     SELECT table_schema AS 'Database',
#             ROUND(SUM(data_length + index_length) / 1024 / 1024, 2) AS 'Size (MB)'
#     FROM information_schema.tables
#     GROUP BY table_schema
#     ORDER BY SUM(data_length + index_length) DESC;
# """)

# # Fetch All Databases Info
# databases = cursor.fetchall()

# # Print the Results
# print("📂 List of Databases and Their Sizes:")
# print("-" * 50)
# for db in databases:
#     print(f"📁 Database: {db[0]} | 🏷️ Size: {db[1]} MB")

# # Switch to SalesDB and Show All Tables
# cursor.execute("USE SalesDB;")  # Switch to SalesDB

# # Show All Tables in SalesDB
# cursor.execute("SHOW TABLES;")
# tables = cursor.fetchall()

# print("\n📋 Tables in 'SalesDB':")
# print("-" * 50)
# for table in tables:
#     table_name = table[0]
#     print(f"📌 {table_name}")

#     # Get All Fields of Each Table
#     cursor.execute(f"SHOW COLUMNS FROM {table_name};")
#     columns = cursor.fetchall()

#     # Print Column Details
#     print(f"   🔍 Fields in `{table_name}`:")
#     print("   " + "-" * 50)
#     for col in columns:
#         print(f"   🏷️ {col[0]:<15} | 📌 Type: {col[1]:<20} | ⚡ Key: {col[3]}")
#     print("")
    
# Close Connection
cursor.close()
conn.close()
